<h1> Predicting Clinical Trial Duration </h1>

---

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

<h2> 1. Feature Selection

In [ ]:
trial_drugbank_df = pd.read_csv('./Datasets/Model Training Data.csv')
drug_approval_predictions_df = pd.read_csv('./Datasets/Drug Approval Predictions.csv')

In [3]:
filtered_trial_drugbank_df = trial_drugbank_df.drop_duplicates(subset = ['NCT ID'])
filtered_drug_approval_predictions_df = drug_approval_predictions_df.drop_duplicates(subset = ['NCT ID'])

Prediction of trial duration only carried out on unique NCT IDs, because one clinical trial tests multiple drugs and are all registered under the same ID, duplicates of the same NCT ID are dropped.

The unseen dataset (drug_approval_predictions) was also cleaned the same way so that they can be applied to the regression model.

In [4]:
filtered_trial_drugbank_df.loc[:, 'Sponsor Type'] = filtered_trial_drugbank_df['Sponsor Type'].map({'INDUSTRY' : 1, 'ACADEMIC' : 2, 'TOP 10 PHARMA' : 3, 'OTHER' : 4, 'CANCER CENTERS' : 5, 'NIH' : 6, 
                                                                           'HOSPITALS' : 7, 'NETWORK' : 8, 'FED' : 9})
filtered_trial_drugbank_df['Sponsor Type'].value_counts()

Sponsor Type
1    17872
2     5981
3     5024
4     2981
5     2735
6     2384
7     2130
8      634
9      300
Name: count, dtype: int64

In [5]:
filtered_trial_drugbank_df.loc[:, 'Medical Condition'] = filtered_trial_drugbank_df['Medical Condition'].map({'Cancer' : 1, 'Other' : 2, 'Infectious Disease' : 3, 'Metabolic/Endocrine' : 4, 'Healthy' : 5, 'Neurological' : 6, 
                                                                                             'Cardiovascular' : 7, 'Autoimmune' : 8, 'Mental Health' : 9, 'Respiratory' : 10, 'Genetic' : 11, 'Digestive' : 12, 
                                                                                             'Ocular' : 13, 'Dermatological' : 14, 'Hematological' : 15, 'Musculoskeletal' : 16})
filtered_trial_drugbank_df['Medical Condition'].value_counts()

Medical Condition
1     13773
2      8192
3      2946
5      2293
4      2155
6      1699
7      1660
8      1489
9      1439
10      972
11      738
12      697
14      576
13      574
15      472
16      366
Name: count, dtype: int64

In [6]:
filtered_trial_drugbank_df.loc[:, 'Trial Status'] = filtered_trial_drugbank_df['Trial Status'].map({'COMPLETED' : 1, 'NOT COMPLETED' : 2, 'IN-PROGRESS' : 3})
filtered_trial_drugbank_df['Trial Status'].value_counts()

Trial Status
1    31155
2     7753
3     1133
Name: count, dtype: int64

In [7]:
filtered_trial_drugbank_df.loc[:, 'Gender'] = filtered_trial_drugbank_df['Gender'].map({'ALL' : 1, 'FEMALE' : 2, 'MALE' : 3})
filtered_trial_drugbank_df['Gender'].value_counts()

Gender
1    35400
2     2719
3     1922
Name: count, dtype: int64

In [8]:
filtered_trial_drugbank_df.loc[:, 'Healthy Participants'] = filtered_trial_drugbank_df['Healthy Participants'].map({True : 1, False : 2})
filtered_trial_drugbank_df['Healthy Participants'].value_counts()

C:\Users\brage\AppData\Local\Temp\ipykernel_19792\1615791418.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 2 2 ... 2 2 2]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  filtered_trial_drugbank_df.loc[:, 'Healthy Participants'] = filtered_trial_drugbank_df['Healthy Participants'].map({True : 1, False : 2})


Healthy Participants
2    32999
1     7042
Name: count, dtype: int64

In [9]:
filtered_trial_drugbank_df.loc[:, 'Phase'] = filtered_trial_drugbank_df['Phase'].map({'PHASE1' : 1, 'PHASE2' : 2, 'PHASE3' : 3})
filtered_trial_drugbank_df['Phase'].value_counts()

Phase
2    17474
1    11968
3    10599
Name: count, dtype: int64

In [10]:
filtered_drug_approval_predictions_df.loc[:, 'Sponsor Type'] = filtered_drug_approval_predictions_df['Sponsor Type'].map({'INDUSTRY' : 1, 'ACADEMIC' : 2, 'TOP 10 PHARMA' : 3, 'OTHER' : 4, 'CANCER CENTERS' : 5, 'NIH' : 6, 
                                                                           'HOSPITALS' : 7, 'NETWORK' : 8, 'FED' : 9})
filtered_drug_approval_predictions_df.loc[:, 'Medical Condition'] = filtered_drug_approval_predictions_df['Medical Condition'].map({'Cancer' : 1, 'Other' : 2, 'Infectious Disease' : 3, 'Metabolic/Endocrine' : 4, 'Healthy' : 5, 'Neurological' : 6, 
                                                                                             'Cardiovascular' : 7, 'Autoimmune' : 8, 'Mental Health' : 9, 'Respiratory' : 10, 'Genetic' : 11, 'Digestive' : 12, 
                                                                                             'Ocular' : 13, 'Dermatological' : 14, 'Hematological' : 15, 'Musculoskeletal' : 16})
filtered_drug_approval_predictions_df.loc[:, 'Trial Status'] = filtered_drug_approval_predictions_df['Trial Status'].map({'COMPLETED' : 1, 'NOT COMPLETED' : 2, 'IN-PROGRESS' : 3})
filtered_drug_approval_predictions_df.loc[:, 'Gender'] = filtered_drug_approval_predictions_df['Gender'].map({'ALL' : 1, 'FEMALE' : 2, 'MALE' : 3})
filtered_drug_approval_predictions_df.loc[:, 'Healthy Participants'] = filtered_drug_approval_predictions_df['Healthy Participants'].map({True : 1, False : 2})
filtered_drug_approval_predictions_df.loc[:, 'Phase'] = filtered_drug_approval_predictions_df['Phase'].map({'PHASE1' : 1, 'PHASE2' : 2, 'PHASE3' : 3, 'PHASE1, PHASE2' : 4, 'PHASE2, PHASE3' : 5})

C:\Users\brage\AppData\Local\Temp\ipykernel_19792\494611543.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 2 ... 1 2 2]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  filtered_drug_approval_predictions_df.loc[:, 'Healthy Participants'] = filtered_drug_approval_predictions_df['Healthy Participants'].map({True : 1, False : 2})


All categorical features are mapped by ordinal encoding. The models that I am using as regression models prefer ordinal encoding. One-hot encoding would hurt model performance because it instills a false sense of feature sparsity.

In [11]:
mask = (filtered_trial_drugbank_df['Phase'] == 1) & (filtered_trial_drugbank_df['Trial Duration (Days)'] <= 3212)
phase1 = filtered_trial_drugbank_df[mask]
phase1['Phase'].value_counts()

Phase
1    11558
Name: count, dtype: int64

In [12]:
mask = (filtered_trial_drugbank_df['Phase'] == 2) & (filtered_trial_drugbank_df['Trial Duration (Days)'] <= 3774)
phase2 = filtered_trial_drugbank_df[mask]
phase2['Phase'].value_counts()

Phase
2    16825
Name: count, dtype: int64

In [13]:
mask = (filtered_trial_drugbank_df['Phase'] == 3) & (filtered_trial_drugbank_df['Trial Duration (Days)'] <= 3073)
phase3 = filtered_trial_drugbank_df[mask]
phase3['Phase'].value_counts()

Phase
3    10044
Name: count, dtype: int64

The training data is split into 3 groups for training separate models to achieve lower RMSE and higher R2 scores for each model: Phase 1, Phase 2, and Phase 3 clinical trial data.

Trial duration outliers, determined from EDA, were removed from the training data to obtain higher performing models.

In [14]:
phase1_unseen = filtered_drug_approval_predictions_df[(filtered_drug_approval_predictions_df['Phase'] == 1)]
phase2_unseen = filtered_drug_approval_predictions_df[(filtered_drug_approval_predictions_df['Phase'] == 2)]
phase3_unseen = filtered_drug_approval_predictions_df[(filtered_drug_approval_predictions_df['Phase'] == 3)]

The same was done for unseen data, except outliers are not removed because the outlier for the unseen data will be very different from that of the training data.

In [15]:
X1 = phase1
X2 = phase2
X3 = phase3

In [16]:
X1_unseen = phase1_unseen
X2_unseen = phase2_unseen
X3_unseen = phase3_unseen

In [17]:
y1 = phase1['Trial Duration (Days)']
y2 = phase2['Trial Duration (Days)']
y3 = phase3['Trial Duration (Days)']

Assign output variable to y and input variables to X.

<h2> 2. Modelling and Hyperparameter Tuning with Decision Tree Regressor

In [18]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, random_state = 99) 
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state = 99) 
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, random_state = 99) 

In [19]:
dtreergr = DecisionTreeRegressor(random_state = 99)
dtreergr.get_params() # default model parameters

{'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'random_state': 99,
 'splitter': 'best'}

I will use the Decision Tree Regressor as my first model.

<h3> 2.1. Optimization of feature list to produce lowest RMSE score

In [20]:
# Feature selection setup
selected_features1 = []
remaining_features = list(['Sponsor Type', 'Trial Status', 'Medical Condition', 'No. of Trial Locations', 'No of Participants', 'Healthy Participants', 'Gender', 
                           'Sponsor_Prior Completed Trials', 'Sponsor_Prior Not Completed Trials'])
best_rmse = np.inf

while remaining_features:
    improved = False
    feature_to_add = None
    rmse_with_feature = best_rmse
    
    for feature in remaining_features:
        trial_features = selected_features1 + [feature]
        dtreergr.fit(X_train1[trial_features], y_train1)
        y_prediction1 = dtreergr.predict(X_test1[trial_features])
        rmse = metrics.root_mean_squared_error(y_test1, y_prediction1)
        
        # Keep track of the best candidate to add
        if rmse < rmse_with_feature:
            rmse_with_feature = rmse
            feature_to_add = feature
            improved = True
    
    if improved:
        selected_features1.append(feature_to_add)
        remaining_features.remove(feature_to_add)
        best_rmse = rmse_with_feature
    else:
        break

print("\nFinal selected features:")
print(selected_features1)
print(f"Final RMSE: {best_rmse:.3f}")


Final selected features:
['Medical Condition', 'Sponsor Type', 'Healthy Participants', 'Trial Status']
Final RMSE: 554.993


In [21]:
X1 = phase1[selected_features1]

In [22]:
# Feature selection setup
selected_features2 = []
remaining_features = list(['Sponsor Type', 'Trial Status', 'Medical Condition', 'No. of Trial Locations', 'No of Participants', 'Healthy Participants', 'Gender', 
                           'Sponsor_Prior Completed Trials', 'Sponsor_Prior Not Completed Trials'])
best_rmse = np.inf

while remaining_features:
    improved = False
    feature_to_add = None
    rmse_with_feature = best_rmse
    
    for feature in remaining_features:
        trial_features = selected_features2 + [feature]
        dtreergr.fit(X_train2[trial_features], y_train2)
        y_prediction2 = dtreergr.predict(X_test2[trial_features])
        rmse = metrics.root_mean_squared_error(y_test2, y_prediction2)
        
        # Keep track of the best candidate to add
        if rmse < rmse_with_feature:
            rmse_with_feature = rmse
            feature_to_add = feature
            improved = True
    
    if improved:
        selected_features2.append(feature_to_add)
        remaining_features.remove(feature_to_add)
        best_rmse = rmse_with_feature
    else:
        break

print("\nFinal selected features:")
print(selected_features2)
print(f"Final RMSE: {best_rmse:.3f}")


Final selected features:
['Sponsor Type', 'Medical Condition', 'Trial Status', 'Healthy Participants']
Final RMSE: 692.864


In [23]:
X2 = phase2[selected_features2]

In [24]:
# Feature selection setup
selected_features3 = []
remaining_features = list(['Sponsor Type', 'Trial Status', 'Medical Condition', 'No. of Trial Locations', 'No of Participants', 'Healthy Participants', 'Gender', 
                           'Sponsor_Prior Completed Trials', 'Sponsor_Prior Not Completed Trials'])
best_rmse = np.inf

while remaining_features:
    improved = False
    feature_to_add = None
    rmse_with_feature = best_rmse
    
    for feature in remaining_features:
        trial_features = selected_features3 + [feature]
        dtreergr.fit(X_train3[trial_features], y_train3)
        y_prediction3 = dtreergr.predict(X_test3[trial_features])
        rmse = metrics.root_mean_squared_error(y_test3, y_prediction3)
        
        # Keep track of the best candidate to add
        if rmse < rmse_with_feature:
            rmse_with_feature = rmse
            feature_to_add = feature
            improved = True
    
    if improved:
        selected_features3.append(feature_to_add)
        remaining_features.remove(feature_to_add)
        best_rmse = rmse_with_feature
    else:
        break

print("\nFinal selected features:")
print(selected_features3)
print(f"Final RMSE: {best_rmse:.3f}")


Final selected features:
['Medical Condition', 'Sponsor Type', 'No. of Trial Locations']
Final RMSE: 615.542


In [25]:
X3 = phase3[selected_features3]

Looped through the entire list of features to select a list of features that has been optimized to give the lowest model RMSE score for all 3 groups of training data: X1, X2, and X3.

In [26]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, random_state = 99) 
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state = 99) 
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, random_state = 99) 

In [27]:
dtreergr.fit(X_train1, y_train1)

DecisionTreeRegressor(random_state=99)

In [28]:
y_prediction1 = dtreergr.predict(X_test1)

rmse = metrics.root_mean_squared_error(y_test1, y_prediction1)
r2 = metrics.r2_score(y_test1, y_prediction1)

phase1_report_df = pd.DataFrame({'Metric': ['RMSE', 'R²'],'phase1_dtree': [f"{rmse:.0f}", f"{r2:.3f}"]})
phase1_report_df

,Metric,phase1_dtree
0,RMSE,555
1,R²,0.464


In [29]:
dtreergr.fit(X_train2, y_train2)

DecisionTreeRegressor(random_state=99)

In [30]:
y_prediction2 = dtreergr.predict(X_test2)

rmse = metrics.root_mean_squared_error(y_test2, y_prediction2)
r2 = metrics.r2_score(y_test2, y_prediction2)

phase2_report_df = pd.DataFrame({'Metric': ['RMSE', 'R²'],'phase2_dtree': [f"{rmse:.0f}", f"{r2:.3f}"]})
phase2_report_df

,Metric,phase2_dtree
0,RMSE,693
1,R²,0.342


In [31]:
dtreergr.fit(X_train3, y_train3)

DecisionTreeRegressor(random_state=99)

In [32]:
y_prediction3 = dtreergr.predict(X_test3)

rmse = metrics.root_mean_squared_error(y_test3, y_prediction3)
r2 = metrics.r2_score(y_test3, y_prediction3)

phase3_report_df = pd.DataFrame({'Metric': ['RMSE', 'R²'],'phase3_dtree': [f"{rmse:.0f}", f"{r2:.3f}"]})
phase3_report_df

,Metric,phase3_dtree
0,RMSE,616
1,R²,0.193


In [33]:
# Calculating null RMSE as the baseline model
y_null1 = np.zeros_like(y_test1, dtype=float)
y_null1.fill(y_train1.mean())

print(f'Phase 1 Null RMSE: {metrics.root_mean_squared_error(y_test1,y_null1):.0f}')

y_null2 = np.zeros_like(y_test2, dtype=float)
y_null2.fill(y_train2.mean())

print(f'Phase 2 Null RMSE: {metrics.root_mean_squared_error(y_test2,y_null2):.0f}')

y_null3 = np.zeros_like(y_test3, dtype=float)
y_null3.fill(y_train3.mean())

print(f'Phase 3 Null RMSE: {metrics.root_mean_squared_error(y_test3,y_null3):.0f}')

Phase 1 Null RMSE: 758
Phase 2 Null RMSE: 854
Phase 3 Null RMSE: 685


All 3 model had lower RMSE scores than their respective null accuracy values. Phase 3 model was the poorest performing model out of the 3.

<h3> 2.2. Hyperparameter tuning of Decision Tree Regressor to produce lowest RMSE

In [34]:
grid_params = {'max_depth': [None] + list(range(5, 15)), 
                'min_samples_split': list(range(2, 20)), 
                'min_samples_leaf': list(range(1, 10)), 
                'max_features': [None, 'sqrt', 'log2'],
                'random_state' : [99]} 

In [35]:
grid_search = GridSearchCV(dtreergr, grid_params, cv = 3, scoring = 'neg_root_mean_squared_error', n_jobs = -1)

In [36]:
grid_search.fit(X_train1, y_train1)

c:\Users\brage\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=99), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                               12, 13, 14, 15, 16, 17, 18, 19],
                         'random_state': [99]},
             scoring='neg_root_mean_squared_error')

Tuning by gridsearchCV to return the hyperparameters that gives the lowest RMSE for the phase 1 model.

In [37]:
print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")

Best Hyperparameters: {'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 18, 'random_state': 99}
Best Score: -533.8583129860209


In [38]:
# Enter best hyperparameters directly into model and fit to avoid re-running GridSearch CV
# dtreergr = DecisionTreeRegressor(max_depth = 12, max_features = None, min_samples_leaf = 9, min_samples_split = 2, random_state = 99)

# dtreergr.fit(X_train, y_train)

In [39]:
y_prediction1 = grid_search.best_estimator_.predict(X_test1)

rmse = metrics.root_mean_squared_error(y_test1, y_prediction1)
r2 = metrics.r2_score(y_test1, y_prediction1)

phase1_report2_df = pd.DataFrame({'Metric': ['RMSE', 'R²'],'phase1_tuned_dtree': [f"{rmse:.0f}", f"{r2:.3f}"]})
phase1_report2_df

,Metric,phase1_tuned_dtree
0,RMSE,543
1,R²,0.486


In [40]:
grid_search.fit(X_train2, y_train2)

GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=99), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                               12, 13, 14, 15, 16, 17, 18, 19],
                         'random_state': [99]},
             scoring='neg_root_mean_squared_error')

Tuning by gridsearchCV to return the hyperparameters that gives the lowest RMSE for the phase 2 model.

In [41]:
y_prediction2 = grid_search.best_estimator_.predict(X_test2)

rmse = metrics.root_mean_squared_error(y_test2, y_prediction2)
r2 = metrics.r2_score(y_test2, y_prediction2)

phase2_report2_df = pd.DataFrame({'Metric': ['RMSE', 'R²'],'phase2_tuned_dtree': [f"{rmse:.0f}", f"{r2:.3f}"]})
phase2_report2_df

,Metric,phase2_tuned_dtree
0,RMSE,694
1,R²,0.339


In [42]:
grid_search.fit(X_train3, y_train3)

GridSearchCV(cv=3, estimator=DecisionTreeRegressor(random_state=99), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                               12, 13, 14, 15, 16, 17, 18, 19],
                         'random_state': [99]},
             scoring='neg_root_mean_squared_error')

Tuning by gridsearchCV to return the hyperparameters that gives the lowest RMSE for the phase 3 model.

In [43]:
y_prediction3 = grid_search.best_estimator_.predict(X_test3)

rmse = metrics.root_mean_squared_error(y_test3, y_prediction3)
r2 = metrics.r2_score(y_test3, y_prediction3)

phase3_report2_df = pd.DataFrame({'Metric': ['RMSE', 'R²'],'phase3_tuned_dtree': [f"{rmse:.0f}", f"{r2:.3f}"]})
phase3_report2_df

,Metric,phase3_tuned_dtree
0,RMSE,587
1,R²,0.268


In [44]:
summary_report_dtree_df = pd.concat([phase1_report_df, phase1_report2_df, phase2_report_df, phase2_report2_df, phase3_report_df, phase3_report2_df], axis = 1)
summary_report_dtree_df = summary_report_dtree_df.drop('Metric', axis = 1)
summary_report_dtree_df = summary_report_dtree_df.rename(index = {0: 'RMSE', 1: 'R²',})
summary_report_dtree_df

,phase1_dtree,phase1_tuned_dtree,phase2_dtree,phase2_tuned_dtree,phase3_dtree,phase3_tuned_dtree
RMSE,555,543,693,694,616,587
R²,0.464,0.486,0.342,0.339,0.193,0.268


Overall, after hyperparameter tuning, the phase 1 model improved slightly for both RMSE and R2 scores. <br>

Phase 2 model performance got slightly worse. <br>

Phase 3 model improved the most for both RMSE and R2 scores.

<h2> 3. Modelling and Hyperparameter Tuning with Random Forest Regressor

In [45]:
from sklearn.ensemble import RandomForestRegressor

In [46]:
X1 = phase1
X2 = phase2
X3 = phase3

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, random_state = 99) 
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state = 99) 
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, random_state = 99) 

In [47]:
rforestrgr = RandomForestRegressor(random_state = 99)
rforestrgr.get_params() # default model parameters

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 99,
 'verbose': 0,
 'warm_start': False}

I will use the Random Forest Regressor as my second model.

<h3> 3.1. Optimization of feature list to produce lowest RMSE

In [48]:
# Feature selection setup
selected_features1 = []
remaining_features = list(['Sponsor Type', 'Trial Status', 'Medical Condition', 'No. of Trial Locations', 'No of Participants', 'Healthy Participants', 'Gender', 
                           'Sponsor_Prior Completed Trials', 'Sponsor_Prior Not Completed Trials'])
best_rmse = np.inf

while remaining_features:
    improved = False
    feature_to_add = None
    rmse_with_feature = best_rmse
    
    for feature in remaining_features:
        trial_features = selected_features1 + [feature]
        rforestrgr.fit(X_train1[trial_features], y_train1)
        y_prediction1 = rforestrgr.predict(X_test1[trial_features])
        rmse = metrics.root_mean_squared_error(y_test1, y_prediction1)
        
        # Keep track of the best candidate to add
        if rmse < rmse_with_feature:
            rmse_with_feature = rmse
            feature_to_add = feature
            improved = True
    
    if improved:
        selected_features1.append(feature_to_add)
        remaining_features.remove(feature_to_add)
        best_rmse = rmse_with_feature
    else:
        break

print("\nFinal selected features:")
print(selected_features1)
print(f"Final RMSE: {best_rmse:.3f}")


Final selected features:
['Medical Condition', 'Sponsor Type', 'Trial Status', 'Healthy Participants']
Final RMSE: 542.483


In [49]:
X1 = phase1[selected_features1]

In [50]:
# Feature selection setup
selected_features2 = []
remaining_features = list(['Sponsor Type', 'Trial Status', 'Medical Condition', 'No. of Trial Locations', 'No of Participants', 'Healthy Participants', 'Gender', 
                           'Sponsor_Prior Completed Trials', 'Sponsor_Prior Not Completed Trials'])
best_rmse = np.inf

while remaining_features:
    improved = False
    feature_to_add = None
    rmse_with_feature = best_rmse
    
    for feature in remaining_features:
        trial_features = selected_features2 + [feature]
        rforestrgr.fit(X_train2[trial_features], y_train2)
        y_prediction2 = rforestrgr.predict(X_test2[trial_features])
        rmse = metrics.root_mean_squared_error(y_test2, y_prediction2)
        
        # Keep track of the best candidate to add
        if rmse < rmse_with_feature:
            rmse_with_feature = rmse
            feature_to_add = feature
            improved = True
    
    if improved:
        selected_features2.append(feature_to_add)
        remaining_features.remove(feature_to_add)
        best_rmse = rmse_with_feature
    else:
        break

print("\nFinal selected features:")
print(selected_features2)
print(f"Final RMSE: {best_rmse:.3f}")


Final selected features:
['Sponsor Type', 'Medical Condition', 'Trial Status', 'Healthy Participants']
Final RMSE: 689.796


In [51]:
X2 = phase2[selected_features2]

In [52]:
# Feature selection setup
selected_features3 = []
remaining_features = list(['Sponsor Type', 'Trial Status', 'Medical Condition', 'No. of Trial Locations', 'No of Participants', 'Healthy Participants', 'Gender', 
                           'Sponsor_Prior Completed Trials', 'Sponsor_Prior Not Completed Trials'])
best_rmse = np.inf

while remaining_features:
    improved = False
    feature_to_add = None
    rmse_with_feature = best_rmse
    
    for feature in remaining_features:
        trial_features = selected_features3 + [feature]
        rforestrgr.fit(X_train3[trial_features], y_train3)
        y_prediction3 = rforestrgr.predict(X_test3[trial_features])
        rmse = metrics.root_mean_squared_error(y_test3, y_prediction3)
        
        # Keep track of the best candidate to add
        if rmse < rmse_with_feature:
            rmse_with_feature = rmse
            feature_to_add = feature
            improved = True
    
    if improved:
        selected_features3.append(feature_to_add)
        remaining_features.remove(feature_to_add)
        best_rmse = rmse_with_feature
    else:
        break

print("\nFinal selected features:")
print(selected_features3)
print(f"Final RMSE: {best_rmse:.3f}")


Final selected features:
['Medical Condition', 'Sponsor Type', 'No. of Trial Locations', 'Healthy Participants']
Final RMSE: 599.857


In [53]:
X3 = phase3[selected_features3]

Looped through the entire list of features to select a list of features that has been optimized to give the lowest model RMSE score for all 3 groups of training data: X1, X2, and X3.

In [54]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, random_state = 99) 
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state = 99) 
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, random_state = 99) 

In [55]:
rforestrgr.fit(X_train1, y_train1)

RandomForestRegressor(random_state=99)

In [56]:
y_prediction1 = rforestrgr.predict(X_test1)

rmse = metrics.root_mean_squared_error(y_test1, y_prediction1)
r2 = metrics.r2_score(y_test1, y_prediction1)

phase1_report3_df = pd.DataFrame({'Metric': ['RMSE', 'R²'],'phase1_rforest': [f"{rmse:.0f}", f"{r2:.3f}"]})
phase1_report3_df

,Metric,phase1_rforest
0,RMSE,542
1,R²,0.487


In [57]:
rforestrgr.fit(X_train2, y_train2)

RandomForestRegressor(random_state=99)

In [58]:
y_prediction2 = rforestrgr.predict(X_test2)

rmse = metrics.root_mean_squared_error(y_test2, y_prediction2)
r2 = metrics.r2_score(y_test2, y_prediction2)

phase2_report3_df = pd.DataFrame({'Metric': ['RMSE', 'R²'],'phase2_rforest': [f"{rmse:.0f}", f"{r2:.3f}"]})
phase2_report3_df

,Metric,phase2_rforest
0,RMSE,690
1,R²,0.348


In [59]:
rforestrgr.fit(X_train3, y_train3)

RandomForestRegressor(random_state=99)

In [60]:
y_prediction3 = rforestrgr.predict(X_test3)

rmse = metrics.root_mean_squared_error(y_test3, y_prediction3)
r2 = metrics.r2_score(y_test3, y_prediction3)

phase3_report3_df = pd.DataFrame({'Metric': ['RMSE', 'R²'],'phase3_rforest': [f"{rmse:.0f}", f"{r2:.3f}"]})
phase3_report3_df

,Metric,phase3_rforest
0,RMSE,600
1,R²,0.234


<h3> 3.2. Hyperparameter tuning of Random Forest Regressor to produce lowest RMSE

In [61]:
grid_params = {'n_estimators': [100],
                'max_depth': [None] + list(range(5, 10)), 
                'min_samples_split': list(range(2, 20)),
                'min_samples_leaf': list(range(1, 5)), 
                'max_features': ['sqrt', 'log2'],
                'random_state' : [99]} 

In [62]:
grid_search = GridSearchCV(rforestrgr, grid_params, cv = 3, scoring = 'neg_root_mean_squared_error', n_jobs = -1)

In [63]:
grid_search.fit(X_train1, y_train1)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=99), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 6, 7, 8, 9],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                               12, 13, 14, 15, 16, 17, 18, 19],
                         'n_estimators': [100], 'random_state': [99]},
             scoring='neg_root_mean_squared_error')

Tuning by gridsearchCV to return the hyperparameters that gives the lowest RMSE for the phase 1 model.

In [64]:
print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")

Best Hyperparameters: {'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 14, 'n_estimators': 100, 'random_state': 99}
Best Score: -531.783654851423


In [65]:
# Enter best hyperparameters directly into model and fit to avoid re-running GridSearch CV
# rforestrgr = RandomForestRegressor(max_depth = None, max_features = 'sqrt', min_samples_leaf = 4, min_samples_split = 19, n_estimators = 100, random_state = 99)

# rforestrgr.fit(X_train, y_train)

In [66]:
y_prediction1 = grid_search.best_estimator_.predict(X_test1)

rmse = metrics.root_mean_squared_error(y_test1, y_prediction1)
r2 = metrics.r2_score(y_test1, y_prediction1)

phase1_report4_df = pd.DataFrame({'Metric': ['RMSE', 'R²'],'phase1_tuned_rforest': [f"{rmse:.0f}", f"{r2:.3f}"]})
phase1_report4_df

,Metric,phase1_tuned_rforest
0,RMSE,534
1,R²,0.504


In [67]:
grid_search.fit(X_train2, y_train2)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=99), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 6, 7, 8, 9],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                               12, 13, 14, 15, 16, 17, 18, 19],
                         'n_estimators': [100], 'random_state': [99]},
             scoring='neg_root_mean_squared_error')

Tuning by gridsearchCV to return the hyperparameters that gives the lowest RMSE for the phase 2 model.

In [68]:
y_prediction2 = grid_search.best_estimator_.predict(X_test2)

rmse = metrics.root_mean_squared_error(y_test2, y_prediction2)
r2 = metrics.r2_score(y_test2, y_prediction2)

phase2_report4_df = pd.DataFrame({'Metric': ['RMSE', 'R²'],'phase2_tuned_rforest': [f"{rmse:.0f}", f"{r2:.3f}"]})
phase2_report4_df

,Metric,phase2_tuned_rforest
0,RMSE,684
1,R²,0.358


In [69]:
grid_search.fit(X_train3, y_train3)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=99), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 6, 7, 8, 9],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                               12, 13, 14, 15, 16, 17, 18, 19],
                         'n_estimators': [100], 'random_state': [99]},
             scoring='neg_root_mean_squared_error')

Tuning by gridsearchCV to return the hyperparameters that gives the lowest RMSE for the phase 3 model.

In [70]:
y_prediction3 = grid_search.best_estimator_.predict(X_test3)

rmse = metrics.root_mean_squared_error(y_test3, y_prediction3)
r2 = metrics.r2_score(y_test3, y_prediction3)

phase3_report4_df = pd.DataFrame({'Metric': ['RMSE', 'R²'],'phase3_tuned_rforest': [f"{rmse:.0f}", f"{r2:.3f}"]})
phase3_report4_df

,Metric,phase3_tuned_rforest
0,RMSE,575
1,R²,0.296


In [71]:
summary_report_rforest_df = pd.concat([phase1_report3_df, phase1_report4_df, phase2_report3_df, phase2_report4_df, phase3_report3_df, phase3_report4_df], axis = 1)
summary_report_rforest_df = summary_report_rforest_df.drop('Metric', axis = 1)
summary_report_rforest_df = summary_report_rforest_df.rename(index = {0: 'RMSE', 1: 'R²',})
summary_report_rforest_df

,phase1_rforest,phase1_tuned_rforest,phase2_rforest,phase2_tuned_rforest,phase3_rforest,phase3_tuned_rforest
RMSE,542,534,690,684,600,575
R²,0.487,0.504,0.348,0.358,0.234,0.296


Overall, after hyperparameter tuning, both the phase 1 and 2 models improved slightly for both RMSE and R2 scores. <br>

Phase 3 model improved the most for both RMSE and R2 scores.

In [72]:
phase1_final_report_df = pd.concat([phase1_report_df, phase1_report2_df, phase1_report3_df, phase1_report4_df], axis = 1)
phase1_final_report_df = phase1_final_report_df.drop('Metric', axis = 1)
phase1_final_report_df = phase1_final_report_df.rename(index = {0: 'RMSE', 1: 'R²',})
phase1_final_report_df

,phase1_dtree,phase1_tuned_dtree,phase1_rforest,phase1_tuned_rforest
RMSE,555,543,542,534
R²,0.464,0.486,0.487,0.504


In [73]:
phase2_final_report_df = pd.concat([phase2_report_df, phase2_report2_df, phase2_report3_df, phase2_report4_df], axis = 1)
phase2_final_report_df = phase2_final_report_df.drop('Metric', axis = 1)
phase2_final_report_df = phase2_final_report_df.rename(index = {0: 'RMSE', 1: 'R²',})
phase2_final_report_df

,phase2_dtree,phase2_tuned_dtree,phase2_rforest,phase2_tuned_rforest
RMSE,693,694,690,684
R²,0.342,0.339,0.348,0.358


In [74]:
phase3_final_report_df = pd.concat([phase3_report_df, phase3_report2_df, phase3_report3_df, phase3_report4_df], axis = 1)
phase3_final_report_df = phase3_final_report_df.drop('Metric', axis = 1)
phase3_final_report_df = phase3_final_report_df.rename(index = {0: 'RMSE', 1: 'R²',})
phase3_final_report_df

,phase3_dtree,phase3_tuned_dtree,phase3_rforest,phase3_tuned_rforest
RMSE,616,587,600,575
R²,0.193,0.268,0.234,0.296


For all phase 1, 2, and 3 models, the random forest models produced better RMSE and R2 scores than their respective decision tree models. <br>

After hyperparameter tuning of the random forest models, all phase 1, 2, and 3 models produced better RMSE and R2 scores. <br>

Therefore, the  tuned random forest models were chosen as my models for predicting trial duration of phase 1, 2, and 3 unseen trial data.

In [75]:
# Assign y_test and y_prediction values to variables for plotting of phase 1 trial duration data
results_df = pd.DataFrame({'Actual': y_test1,'Predicted': y_prediction1})

# Plot of predicted vs actual trial duration
fig1 = px.scatter(results_df, x = 'Actual', y = 'Predicted', opacity = 0.6, width = 600,
                 labels = {'Actual': 'Actual Trial Duration (Days)', 'Predicted': 'Predicted Trial Duration (Days)'})

# Add diagonal perfect prediction line
min_val = min(results_df.min())
max_val = max(results_df.max())
fig1.add_trace(go.Scatter(mode = 'lines',
               x = [min_val, max_val],
               y = [min_val, max_val],
               line = dict(color = 'red', dash = 'dash'),
               name = 'Perfect Prediction'))

fig1.update_layout(title = {'text' : 'Predicted vs Actual Values (Phase 1)', 'x' : 0.5, 'y' : 0.95, 'xanchor' : 'center'})
fig1.update_layout(legend = dict(x = 0, y = 1, xanchor = 'left', yanchor = 'top', bgcolor = 'rgba(255, 255, 255, 0.5)'))
fig1.show()

In [76]:
# Assign y_test and y_prediction values to variables for plottingof phase 2 trial duration data
results_df = pd.DataFrame({'Actual': y_test2,'Predicted': y_prediction2})

# Plot of predicted vs actual trial duration
fig2 = px.scatter(results_df, x = 'Actual', y = 'Predicted', opacity=0.6, width = 600,
                 labels={'Actual': 'Actual Trial Duration (Days)', 'Predicted': 'Predicted Trial Duration (Days)'})

# Add diagonal perfect prediction line
min_val = min(results_df.min())
max_val = max(results_df.max())
fig2.add_trace(go.Scatter(mode='lines',
               x = [min_val, max_val],
               y = [min_val, max_val],
               line = dict(color = 'red', dash = 'dash'),
               name = 'Perfect Prediction'))

fig2.update_layout(title = {'text' : 'Predicted vs Actual Values (Phase 2)', 'x' : 0.5, 'y' : 0.95, 'xanchor' : 'center'})
fig2.update_layout(legend = dict(x = 0, y = 1, xanchor = 'left', yanchor = 'top', bgcolor = 'rgba(255, 255, 255, 0.5)'))
fig2.show()

In [77]:
# Assign y_test and y_prediction values to variables for plottingof phase 3 trial duration data
results_df = pd.DataFrame({'Actual': y_test3,'Predicted': y_prediction3})

# Plot of predicted vs actual trial duration
fig3 = px.scatter(results_df, x = 'Actual', y = 'Predicted', opacity = 0.6, width = 600,
                 labels={'Actual': 'Actual Trial Duration (Days)', 'Predicted': 'Predicted Trial Duration (Days)'})

# Add diagonal perfect prediction line
min_val = min(results_df.min())
max_val = max(results_df.max())
fig3.add_trace(go.Scatter(mode='lines',
               x = [min_val, max_val],
               y = [min_val, max_val],
               line = dict(color = 'red', dash = 'dash'),
               name ='Perfect Prediction'))

fig3.update_layout(title = {'text' : 'Predicted vs Actual Values (Phase 3)', 'x' : 0.5, 'y' : 0.95, 'xanchor' : 'center'})
fig3.update_layout(legend = dict(x = 0, y = 1, xanchor = 'left', yanchor = 'top', bgcolor = 'rgba(255, 255, 255, 0.5)'))
fig3.show()

Unfortunately, comparing the actual vs predicted trial daration for phase 1, 2, and 3 clinical trials, the predicted data deviates quite significantly from the actual data. <br>

This could be because the trial duration variable is highly irregular in nature and no real relationship can be established with the features in our model. <br>

More work needs to be done in the future to examine new features outside of our current dataset to develop a more accurate regression model for predicting clinical trial duration.

<h2> 4. Predicting Trial Duration of Clinical Trial Data under Investigational Status

In [78]:
X1 = phase1[selected_features1]
X2 = phase2[selected_features2]
X3 = phase3[selected_features3]

In [79]:
X1_unseen = phase1_unseen[selected_features1]
X2_unseen = phase2_unseen[selected_features2]
X3_unseen = phase3_unseen[selected_features3]

The feature lists optimized to produce the lowest RMSE were selected for both the training and unseen data.

In [80]:
grid_params = {'n_estimators': [100],
                'max_depth': [None] + list(range(5, 10)), 
                'min_samples_split': list(range(2, 20)),
                'min_samples_leaf': list(range(1, 5)), 
                'max_features': ['sqrt', 'log2'],
                'random_state' : [99]} 

In [81]:
grid_search = GridSearchCV(rforestrgr, grid_params, cv = 3, scoring = 'neg_root_mean_squared_error', n_jobs = -1)

In [82]:
grid_search.fit(X1, y1)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=99), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 6, 7, 8, 9],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                               12, 13, 14, 15, 16, 17, 18, 19],
                         'n_estimators': [100], 'random_state': [99]},
             scoring='neg_root_mean_squared_error')

The entire dataset, not just X_train and y_train, was fitted onto the random forest model optimized for lowest RMSE for the phase 1 model.

In [83]:
y_prediction1_unseen = grid_search.best_estimator_.predict(X1_unseen)
y_prediction1_unseen

array([432.3521589 , 178.78665216, 173.76624308, ..., 784.15851633,
       128.00008694, 822.91606604])

In [84]:
grid_search.fit(X2, y2)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=99), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 6, 7, 8, 9],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                               12, 13, 14, 15, 16, 17, 18, 19],
                         'n_estimators': [100], 'random_state': [99]},
             scoring='neg_root_mean_squared_error')

The entire dataset, not just X_train and y_train, was fitted onto the random forest model optimized for lowest RMSE for the phase 2 model.

In [85]:
y_prediction2_unseen = grid_search.best_estimator_.predict(X2_unseen)
y_prediction2_unseen

array([1250.84692178,  768.64055058, 1924.52322751, ..., 2103.89182176,
       2103.89182176, 2103.89182176])

In [86]:
grid_search.fit(X3, y3)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=99), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 6, 7, 8, 9],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                               12, 13, 14, 15, 16, 17, 18, 19],
                         'n_estimators': [100], 'random_state': [99]},
             scoring='neg_root_mean_squared_error')

The entire dataset, not just X_train and y_train, was fitted onto the random forest model optimized for lowest RMSE for the phase 3 model.

In [87]:
y_prediction3_unseen = grid_search.best_estimator_.predict(X3_unseen)
y_prediction3_unseen

array([1683.51668614, 2120.99360874,  947.77400526,  443.5276886 ,
        828.49001369,  925.03362794, 1090.92976954,  999.64335713,
        731.3283438 , 1264.32970476,  623.02061125, 2041.08896424,
        984.60609576,  766.0014481 ,  534.12062785,  737.52634195,
       1876.1797361 , 1287.63998084,  884.46854431, 1358.02649757,
        828.49001369, 1264.01328803,  683.52420313, 1229.18528986,
        924.67967779, 1294.92076236, 1223.44756452, 1548.50080197,
       1018.58896952,  889.09286738,  418.28281409,  977.25880193,
       1282.5650806 ,  734.99936927, 1111.59644785,  704.03374497,
       1067.05949756,  534.12062785,  792.71280358, 1419.80941332,
       1891.70203521, 1026.09947481,  734.99936927, 1393.03849436,
        707.81075591, 1062.01376789, 1030.57686726, 1975.16638297,
        913.8023241 , 1130.93020055,  952.02177205, 1345.69875087,
        974.4910127 ,  443.5276886 ,  732.87326214, 1026.09947481,
       1312.91385425,  971.19294253, 1030.57686726,  713.75256

In [88]:
phase1_unseen['Predicted Trial Duration'] = y_prediction1_unseen
phase2_unseen['Predicted Trial Duration2'] = y_prediction2_unseen
phase3_unseen['Predicted Trial Duration3'] = y_prediction3_unseen + 365
phase1_unseen = phase1_unseen[['NCT ID', 'Predicted Trial Duration']]
phase2_unseen = phase2_unseen[['NCT ID', 'Predicted Trial Duration2']]
phase3_unseen = phase3_unseen[['NCT ID', 'Predicted Trial Duration3']]

trial_duration_predictions_df = pd.merge(drug_approval_predictions_df, phase1_unseen, on = 'NCT ID', how = 'left')
trial_duration_predictions_df = pd.merge(trial_duration_predictions_df, phase2_unseen, on = 'NCT ID', how = 'left')
trial_duration_predictions_df = pd.merge(trial_duration_predictions_df, phase3_unseen, on = 'NCT ID', how = 'left')

trial_duration_predictions_df['Predicted Trial Duration'] = trial_duration_predictions_df['Predicted Trial Duration'].fillna(trial_duration_predictions_df['Predicted Trial Duration2'])
trial_duration_predictions_df['Predicted Trial Duration'] = trial_duration_predictions_df['Predicted Trial Duration'].fillna(trial_duration_predictions_df['Predicted Trial Duration3'])
trial_duration_predictions_df = trial_duration_predictions_df.drop(['Predicted Trial Duration2', 'Predicted Trial Duration3'], axis = 1)
trial_duration_predictions_df

C:\Users\brage\AppData\Local\Temp\ipykernel_19792\1019405049.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brage\AppData\Local\Temp\ipykernel_19792\1019405049.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brage\AppData\Local\Temp\ipykernel_19792\1019405049.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

,Unnamed: 0,NCT ID,Trial Status,Phase,Start Date,Completion Date,Trial Duration (Days),Trial Location,Sponsor Name,Sponsor Type,...,Total FED Sponsors per Drug,Total HOSPITALS Sponsors per Drug,Total INDUSTRY Sponsors per Drug,Total NETWORK Sponsors per Drug,Total NIH Sponsors per Drug,Total OTHER Sponsors per Drug,Total TOP 10 PHARMA Sponsors per Drug,No. of Medical Conditions Tested per Drug,Predicted Status,Predicted Trial Duration
0,3,NCT02922166,IN-PROGRESS,PHASE1,2017-02-01,2019-12-01,1033,United States,Azevan Pharmaceuticals,INDUSTRY,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,3,approved,432.352159
1,4,NCT06530966,IN-PROGRESS,PHASE1,2024-07-01,2024-12-01,153,United States,InnoCare Pharma Inc.,INDUSTRY,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,approved,178.786652
2,10,NCT03602066,NOT COMPLETED,PHASE2,2019-02-01,2022-11-01,1369,United States,University of Arizona,ACADEMIC,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,not approved,1250.846922
3,11,NCT03188666,COMPLETED,PHASE2,2018-02-01,2021-09-01,1308,"United States, Canada, France, Italy, Netherla...",Regeneron Pharmaceuticals,INDUSTRY,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,not approved,768.640551
4,13,NCT03101566,COMPLETED,PHASE2,2017-09-01,2021-06-01,1369,United States,University of Michigan Rogel Cancer Center,ACADEMIC,...,0.0,68.0,119.0,42.0,25.0,70.0,68.0,4,approved,1924.523228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14256,84029,NCT01036087,COMPLETED,PHASE2,2010-11-01,2022-08-01,4291,United States,M.D. Anderson Cancer Center,CANCER CENTERS,...,0.0,6.0,23.0,4.0,3.0,8.0,17.0,3,not approved,2103.891822
14257,84032,NCT04998487,COMPLETED,PHASE1,2021-08-01,2022-07-01,334,United States,Nektar Therapeutics,INDUSTRY,...,0.0,0.0,9.0,0.0,0.0,0.0,0.0,4,approved,128.000087
14258,84047,NCT01682187,COMPLETED,PHASE1,2005-12-01,2024-09-01,6849,"United States, Spain",Eli Lilly and Company,TOP 10 PHARMA,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3,approved,822.916066
14259,84048,NCT01682187,COMPLETED,PHASE1,2005-12-01,2024-09-01,6849,"United States, Spain",Eli Lilly and Company,TOP 10 PHARMA,...,0.0,2.0,3.0,6.0,0.0,1.0,6.0,2,approved,822.916066


In [89]:
mask = (trial_duration_predictions_df['Predicted Status'] == 'approved') & (trial_duration_predictions_df['Phase'].isin(['PHASE1', 'PHASE2']))
trial_duration_predictions_df.loc[mask, 'Status'] = 'Trial Completed'

mask = (trial_duration_predictions_df['Predicted Status'] == 'approved') & (trial_duration_predictions_df['Phase'].isin(['PHASE3']))
trial_duration_predictions_df.loc[mask, 'Status'] = 'Drug Approved'

trial_duration_predictions_df['Status'].value_counts()

Status
Trial Completed    4839
Drug Approved       138
Name: count, dtype: int64

Create a new 'status' column to differentiate between phase 1 and 2 trials where predictions will be for completion of trials. For phase 3 trials predictions will be for drug approvals.

In [ ]:
trial_duration_predictions_df.to_csv('./Datasets/Drug Approval and Trial Duration Predictions.csv', index = False)

The predictions for clinical trial durations were added to the unseen dataset and exported.

The final objective will be to use these predictions to generate a forecast of approved drugs from 2025 to 2028.

In [91]:
# fig1.write_image('./Exported figures/fig3.1.png', scale = 3)
# fig2.write_image('./Exported figures/fig3.2.png', scale = 3)
# fig3.write_image('./Exported figures/fig3.3.png', scale = 3)